In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [3]:
import polars as pl

splits = {
    "testmini": "data/testmini-00000-of-00001-725687bf7a18d64b.parquet",
    "test": "data/test-*.parquet",
}
df = pl.read_parquet("hf://datasets/AI4Math/MathVista/" + splits["testmini"]).sample(
    fraction=1.0, shuffle=True, seed=42
)

In [4]:
from typing import Iterator, TypedDict, cast


class DecodedImage(TypedDict):
    bytes: bytes


class Scenario(TypedDict):
    pid: int
    question: str
    answer: str
    image: str
    decoded_image: DecodedImage


val_scenarios = cast(list[Scenario], df.head(64).to_dicts())
train_scenarios_iter = cast(Iterator[Scenario], df.tail(-64).iter_rows(named=True))

In [5]:
import re

import art
from art.local import LocalBackend

model = art.TrainableModel(
    name="001",
    project="math-vista",
    base_model="Qwen/Qwen2.5-VL-7B-Instruct",
)
backend = LocalBackend()
await model.register(backend)
client = model.openai_client()

wandb: Currently logged in as: bhilton (wandb) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


INFO 10-14 22:12:56 [__init__.py:235] Automatically detected platform cuda.


/home/sky/sky_workdir/src/art/__init__.py:10: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  import unsloth  # type: ignore # noqa: F401


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 10-14 22:13:03 [__init__.py:235] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.9.6: Fast Qwen2_5_Vl patching. Transformers: 4.53.2. vLLM: 0.10.0.
   \\   /|    NVIDIA H200. Num GPUs = 1. Max memory: 139.811 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 9.0. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Patching vLLM v1 graph capture
Unsloth: Patching vLLM v0 graph capture
Unsloth: Vision model detected, setting approx_max_num_seqs to 1
Unsloth: vLLM loading unsloth/qwen2.5-vl-7b-instruct-unsloth-bnb-4bit with actual GPU utilization = 78.66%
Unsloth: Your GPU has CUDA compute capability 9.0 with VRAM = 

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00, 34.24it/s]

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.03it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.03it/s]



INFO 10-14 22:13:42 [punica_selector.py:19] Using PunicaWrapperGPU.
INFO 10-14 22:13:42 [model_runner.py:1115] Model loading took 6.5306 GiB and 19.065715 seconds
INFO 10-14 22:13:54 [worker.py:295] Memory profiling takes 11.32 seconds
INFO 10-14 22:13:54 [worker.py:295] the current vLLM instance can use total_gpu_memory (139.81GiB) x gpu_memory_utilization (0.79) = 109.98GiB
INFO 10-14 22:13:54 [worker.py:295] model weights take 6.53GiB; non_torch_memory takes 0.16GiB; PyTorch activation peak memory takes 5.01GiB; the rest of the memory reserved for KV Cache is 98.27GiB.
INFO 10-14 22:13:54 [executor_base.py:113] # cuda blocks: 115005, # CPU blocks: 7021
INFO 10-14 22:13:54 [executor_base.py:118] Maximum concurrency for 32768 tokens per request: 56.15x
INFO 10-14 22:13:57 [vllm_utils.py:721] Unsloth: Running patched vLLM v0 `capture_model`.
INFO 10-14 22:13:57 [model_runner.py:1385] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static.

Capturing CUDA graph shapes: 100%|██████████| 1/1 [00:02<00:00,  2.82s/it]


INFO 10-14 22:14:00 [model_runner.py:1537] Graph capturing finished in 3 secs, took 0.06 GiB
INFO 10-14 22:14:00 [vllm_utils.py:728] Unsloth: Patched vLLM v0 graph capture finished in 3 secs.
INFO 10-14 22:14:00 [llm_engine.py:424] init engine (profile, create kv cache, warmup model) took 17.86 seconds
Unsloth: Just some info: will skip parsing ['layer_norm1', 'input_layernorm', 'post_layernorm', 'norm1', 'q_norm', 'post_feedforward_layernorm', 'layer_norm2', 'pre_feedforward_layernorm', 'post_attention_layernorm', 'k_norm', 'norm2']
Unsloth: Just some info: will skip parsing ['layer_norm1', 'input_layernorm', 'post_layernorm', 'q_norm', 'post_feedforward_layernorm', 'cross_attn_input_layernorm', 'layer_norm2', 'pre_feedforward_layernorm', 'post_attention_layernorm', 'k_norm', 'cross_attn_post_attention_layernorm']


In [6]:
async def rollout(scenario: Scenario) -> art.Trajectory:
    image_path = f"/tmp/{scenario['image']}"

    import os

    os.makedirs(os.path.dirname(image_path), exist_ok=True)

    with open(image_path, "wb") as f:
        f.write(scenario["decoded_image"]["bytes"])

    trajectory = art.Trajectory(messages_and_choices=[], reward=0.0)
    trajectory.messages_and_choices = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": scenario["question"]
                    + "\n\nNote: Provide your answer in a LaTeX box.",
                },
                {"type": "image_url", "image_url": {"url": f"file://{image_path}"}},
            ],
        }
    ]
    chat_completion = await client.chat.completions.create(
        model=model.name, messages=trajectory.messages()
    )
    choice = chat_completion.choices[0]
    trajectory.messages_and_choices.append(choice)
    content = choice.message.content
    assert content is not None
    if matches := list(re.finditer(r"\\boxed\{(.*?)\}", content, re.DOTALL)):
        match = matches[-1]
        answer = match.group(1)
        if answer.lower() == scenario["answer"].lower():
            trajectory.reward = 1.0
    return trajectory

In [7]:
import asyncio
import itertools

SCENARIOS_PER_STEP = 8
TRAJECTORY_GROUP_SIZE = 8
start = await model.get_step()
train_scenarios_iter = itertools.cycle(train_scenarios_iter)
for _ in range(start * SCENARIOS_PER_STEP):
    next(train_scenarios_iter)

for i in range(start, 1000):
    train_scenarios = [next(train_scenarios_iter) for _ in range(SCENARIOS_PER_STEP)]
    val_trajectories, train_trajectory_groups = await asyncio.gather(
        art.gather_trajectories(
            (rollout(scenario) for scenario in val_scenarios),
            pbar_desc="gather(val)",
            max_exceptions=32,
        ),
        art.gather_trajectory_groups(
            (
                art.TrajectoryGroup(
                    rollout(scenario) for _ in range(TRAJECTORY_GROUP_SIZE)
                )
                for scenario in train_scenarios
            ),
            pbar_desc="gather(train)",
            max_exceptions=32,
        ),
    )
    await model.log(val_trajectories)
    break
    await model.train(train_trajectory_groups)

gather(val):   0%|          | 0/64 [00:00<?, ?it/s]

gather(train):   0%|          | 0/64 [00:00<?, ?it/s]

In [ ]:
for item in (
    train_trajectory_groups[0].trajectories[0].messages_and_choices[0]["content"]
):
    print(item)

In [ ]:
await model.train(train_trajectory_groups)

In [ ]:
from art.transformers.patches import patch_preprocess_mask_arguments

patch_preprocess_mask_arguments()

In [ ]:
val_trajectories = await art.gather_trajectories(
    (rollout(scenario) for scenario in val_scenarios)
)
await model.log(val_trajectories)

In [ ]:
val_trajectories[0].messages_and_choices[-1].message.content

In [ ]:
list(
    re.finditer(
        r"\\boxed\{(.*?)\}",
        val_trajectories[0].messages_and_choices[-1].message.content,
        re.DOTALL,
    )
)[-1].group(1)